In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display


In [6]:
# スライダーの作成
x_slider = widgets.FloatSlider(description='X', min=-10.0, max=10.0, value=0)
stick_input_slider = widgets.FloatSlider(description='Stick Input', min=-1.0, max=1.0, value=0)

def update_x(change):
    step = 0.1  # X座標の変化するステップの大きさ
    threshold = 0.5  # スティック入力の閾値

    slide_x = stick_input_slider.value
    if change['new'] > threshold:
        if (slide_x > 0) and (slide_x < 1.0):
            x_slider.value = min(x_slider.max, x_slider.value + step)
        elif slide_x == 1.0 :
            x_slider.value += step
            
    elif change['new'] < -threshold:
        if (slide_x < 0) and (slide_x > -1.0):
            x_slider.value = max(x_slider.min, x_slider.value - step)
        elif slide_x == -1.0:
            x_slider.value -= step

# スティック入力の変化を監視
stick_input_slider.observe(update_x, names='value')

display(x_slider, stick_input_slider)


FloatSlider(value=0.0, description='X', max=10.0, min=-10.0)

FloatSlider(value=0.0, description='Stick Input', max=1.0, min=-1.0)

In [8]:
import ipywidgets as widgets
from IPython.display import display
import threading

# スライダーの作成
x_slider = widgets.FloatSlider(description='X', min=-10.0, max=10.0, value=0)
stick_input_slider = widgets.FloatSlider(description='Stick Input', min=-1.0, max=1.0, value=0)


# X座標を更新する関数
def update_x():
    step = 0.1  # X座標の変化するステップの大きさ
    if stick_input_slider.value == 1.0:
        x_slider.value = min(x_slider.max, x_slider.value + step)
    elif stick_input_slider.value == -1.0:
        x_slider.value = max(x_slider.min, x_slider.value - step)

    # 定期的にこの関数を再度呼び出す
    threading.Timer(0.01, update_x).start()

# 最初の更新を開始
update_x()

display(x_slider, stick_input_slider)


FloatSlider(value=0.0, description='X', max=10.0, min=-10.0)

FloatSlider(value=0.0, description='Stick Input', max=1.0, min=-1.0)

In [10]:
import ipywidgets as widgets
from IPython.display import display
import threading

# スライダーの作成
x_slider = widgets.FloatSlider(description='X', min=-10.0, max=10.0, value=0)
stick_input_slider = widgets.FloatSlider(description='Stick Input', min=-1.0, max=1.0, value=0)


# X座標を更新する関数
def update_x():
    step = 0.1  # X座標の変化するステップの大きさ
    x_change = stick_input_slider.value * step
    x_slider.value = max(min(x_slider.max, x_slider.value + x_change), x_slider.min)

    # 定期的にこの関数を再度呼び出す
    threading.Timer(0.1, update_x).start()

# 最初の更新を開始
update_x()

display(x_slider, stick_input_slider)


FloatSlider(value=0.0, description='X', max=10.0, min=-10.0)

FloatSlider(value=0.0, description='Stick Input', max=1.0, min=-1.0)

In [1]:
import ipywidgets as widgets
from IPython.display import display
import threading

# スライダーの作成
x_slider = widgets.FloatSlider(description='X', min=-10.0, max=10.0, value=0,step = 0.01)
stick_input_slider = widgets.FloatSlider(description='Stick Input', min=-1.0, max=1.0, value=0,step=0.01)
dead_zone_slider = widgets.FloatSlider(description='Dead Zone', min=0.0, max=1.0, value=0.1,step=0.01)
step_slider = widgets.FloatSlider(description='Step Size', min=0.01, max=1.0, value=0.1,step = 0.01)


# X座標を更新する関数
def update_x():
    dead_zone = dead_zone_slider.value
    step = step_slider.value
    x_change = 0

    if abs(stick_input_slider.value) > dead_zone:
        x_change = stick_input_slider.value * step

    x_slider.value = max(min(x_slider.max, x_slider.value + x_change), x_slider.min)

    # 定期的にこの関数を再度呼び出す
    threading.Timer(0.1, update_x).start()

# 最初の更新を開始
update_x()

option = widgets.VBox([dead_zone_slider, step_slider])
position = widgets.VBox([x_slider, stick_input_slider])

display(widgets.HBox([option,position]))
